## 2022 HDAT-DA (Hyundai motor group Data Analyst test) 해설

#### 1. 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
train = pd.read_csv('./train.csv').drop(columns=['ID'])
train_x = train.drop(columns=['Class'])
train_y = train['Class']

test_x = pd.read_csv('./test.csv').drop(columns=['ID'])

#### 2. 기초 통계 분석 & EDA

변수들간 상관관계를 보여주는 pairplot, 연속형 변수의 분포를 보여주는 distplot, 그리고 범주의 변수의 빈도를 보여주는 countplot을 이용해서 EDA를 진행

In [ ]:
plt.figure(figsize = (15,15))
sns.pairplot(train[['V0', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'CLASS']])
plt.show()

#변수들간 선형관계가 없는 것을 확인할 수 있다.

In [ ]:
fig, axes = plt.subplot(3,3,figsize=(20,20))

sns.distplot(x = train_x['V0'], ax=axes[0][0].set_title('V0'))
sns.distplot(x = train_x['V1'], ax=axes[0][1].set_title('V1'))
sns.distplot(x = train_x['V2'], ax=axes[0][2].set_title('V2'))
sns.distplot(x = train_x['V3'], ax=axes[1][0].set_title('V3'))
sns.distplot(x = train_x['V4'], ax=axes[1][1].set_title('V4'))
sns.distplot(x = train_x['V5'], ax=axes[1][2].set_title('V5'))
sns.distplot(x = train_x['V6'], ax=axes[2][0].set_title('V6'))
sns.distplot(x = train_x['V7'], ax=axes[2][1].set_title('V7'))
sns.distplot(x = train_x['CLASS'], ax=axes[2][2].set_title('CLASS'))

plt.show()

#V0~V7의 경우, 분포가 중앙으로 몰려있는 상태
#Class의 경우에는 큰 불균형 없음

#### 3. 데이터 전처리

In [ ]:
#학습 데이터에 결측치가 존재하는지 확인합니다.
for col in train_x.columns:
    col_by_null = train_x[col].isnull().sum()
    print(f'Columns : [{col}] 결측치 개수 : [{col_by_null}]')
    if col_by_null > 0:
        # 학습데이터에 해당 column에 결측치가 1개 이상 존재하는 경우
        # 학습데이터의 해당 column의 평균으로 결측치 대체 - 변수들의 분포가 중앙으로 몰려있는 형태이기 때문
        train_x[col] = train_x[col].fillna(train_x[col].mean())

        # 테스트데이터 역시 결측치가 존재할 수 있기 때문에, 학습데이터의 해당 column의 평균으로 동일하게 결측치 대체
        # [중요] 단, 테스트 데이터의 통계 정보(column별 평균)를 활용하는 것은 Data Leakage에 해당하므로 주의 -> 무조건 학습데이터의 통계량을 활용
        test_x[col] = test_x[col].fillna(train_x[col].mean())
        print('결측치 대체 완료')

    else:
        print('결측치 없음')

#### 4. 분석 모델 설계

선형 모델인 Logistic Regression, 트리기반 모델 중 배깅 기법을 적용한 Random Forest, 트리기반 모델 중 부스팅 기법을 적용한 XGBoost와 LightGBM을 사용하여 어떤 모델이 좋은 성능을 보여주는지 실험합니다.

In [ ]:
lr = LogisticRegression()
rf = RandomForestClassifier()
xgb = XGBClassifier()
lgbm = LGBMClassifier()

4-1. 분석 모델 성능 평가<br>
성능을 평가하기 위해 학습 데이터셋에서 20%를 활용하여 검증 데이터셋을 생성합니다.

In [ ]:
train_x_splited, val_x, train_y_splited, val_y = train_test_split(train_x, train_y, test_size=0.2)

In [ ]:
lr.fit(train_x_splited, train_y_splited)
lr_pred = lr.predict(val_x)
print('Logistic Regression 모델의 정확도 : ', accuracy_score(val_y, lr_pred))

rf.fit(train_x_splited, train_y_splited)
rf_pred = rf.predict(val_x)
print('Random Forest 모델의 정확도 : ', accuracy_score(val_y, rf_pred))

xgb.fit(train_x_splited, train_y_splited)
xgb_pred = xgb.predict(val_x)
print('XGBoost 모델의 정확도 : ', accuracy_score(val_y, xgb_pred))

lgbm.fit(train_x_splited, train_y_splited)
lgbm_pred = lgbm.predict(val_x)
print('LGBM 모델의 정확도 : ', accuracy_score(val_y, lgbm_pred))

4-2. 1차 Gridsearch Cross Validation <br>
이제 검증 데이터셋을 이용한 평가에서 가장 좋은 성능을 보인 LGBM 모델의 하이퍼 파라미터 튜닝을 진행하기 위해 Gridsearch CV를 활용합니다. <br>
다만 너무 많은 조합을 한번에 진행함녀 시간이 너무 오래 걸리기 때문에 먼저 learning_rate와 max_depth 변화에 따른 성능 변화를 살펴봅니다.

In [ ]:
param_grid = [
    {'n_estimators': [100],
     'num_leaves': [100],
     'learning_rate': [0.01, 0.05, 0.1],
     'max_depth': [-1,5,15]}
]

grid_search = GridSearchCV(lgbm,
                           param_grid,
                           cv=3,
                           scoring='accuracy',
                           return_train_score=True,
                           n_jobs=-1)

grid_search.fit(train_x, train_y)

print('최적의 하이퍼 파라미터 : ', grid_search.best_params_)

1차적으로 진행한 Gridsearch CV 결과를 살펴보면, learning rate 0.05에서 가장 좋은 성능을 보이고, max_depth의 경우 -1, 즉 제한이 없을 때 가장 좋은 성능을 보입니다. <br>
이제 이 값을 고정시켜 최적 n_estimators와 num_leaves값을 탐색합니다.

4-3. 2차 Gridsearch Cross Validation <br>
먼저 넓은 범위의 값을 부여해서 성능 향상이 확인되는 구간을 확인합니다.

In [ ]:
param_grid = [
    {'n_estimators': [100, 200, 300, 400],
     'num_leaves': [100, 200, 300, 400],
     'learning_rate': [0.05],
     'max_depth': [-1]}
]

grid_search = GridSearchCV(lgbm,
                           param_grid,
                           cv=3,
                           scoring='accuracy',
                           return_train_score=True,
                           n_jobs=-1)

grid_search.fit(train_x, train_y)

print('최적의 하이퍼 파라미터 : ', grid_search.best_params_)

n_estimators의 경우 300을 중심으로 200과 400 사이에서 성능향상이 확인되며 num_leaves의 경우 100을 중심으로 200미만에서 성능 향상이 확인됩니다.

4-4. 최종 Gridsearch Cross Validation <br>
이제 마지막으로 앞서 찾은 구간을 바탕으로 해당 범위의 값을 세분화해서 최적의 성능을 가진 하이퍼 파라미터를 탐색합니다.

In [ ]:
param_grid = [
    {'n_estimators': [225, 250, 275, 300, 325, 350, 375],
     'num_leaves': [50, 75, 100, 125, 150, 175],
     'learning_rate': [0.05],
     'max_depth': [-1]}
]

grid_search = GridSearchCV(lgbm,
                           param_grid,
                           cv=3,
                           scoring='accuracy',
                           return_train_score=True,
                           n_jobs=-1)

grid_search.fit(train_x, train_y)

print('최적의 하이퍼 파라미터 : ', grid_search.best_params_)
best_params = grid_search.best_params_ # 최적의 하이퍼파라미터 저장

#### 5. 모델 학습

In [ ]:
lgbm = LGBMClassifier(**best_params) # 최적의 하이퍼파라미터를 불러와 모델을 구성
lgbm.fit(train_x, train_y)

#### 6. 예측값 생성

In [ ]:
pred = lgbm.predict(test_x)

#### 7. 제출 파일 생성

In [ ]:
submission = pd.read_csv('./sample_submission.csv')
submission.head(5)

In [ ]:
submission['Class'] = pred
submission.head(5)

In [ ]:
submission.to_csv('./submission.csv', index=False)

#### 8. 성능 평가

In [ ]:
import pandas as pd
import numpy as np

def ACCURACY(true_df, pred_df):
    target_idx = true_df.iloc[:,0]
    pred_df = pred_df[pred_df.iloc[:,0].isin(target_idx)]
    pred_df = pred_df.sort_values(by=[pred_df.columns[0]], ascending=[True])
    true_df = true_df.sort_values(by=[true_df.columns[0]], ascending=[True])

    true = true_df.iloc[:,1].to_numpy()
    pred = pred_df.iloc[:,1].to_numpy()

    score = np.mean(true == pred)

    return score

In [ ]:
answer = pd.read_csv('./solution.csv')
submit = pd.read_csv('./submission.csv')

print(ACCURACY(answer, submit))